### Autograd:自动微分　　
`autograd`包是PyTorch中神经网络的核心, 它可以为基于`tensor`的的所有操作提供自动微分的功能, ??这是一个逐个运行的框架, 意味着反向传播是根据你的代码来运行的, 并且每一次的迭代运行都可能不同  

### Variable  

`autograd.Variable`包裹着`Tensor`, 支持几乎所有`Tensor`的操作,并附加额外的属性, 在进行操作以后, 通过调用`.backward()`来计算梯度, 通过`.data`来访问原始`raw data (tensor)`, 并将变量梯度累加到`.grad`

　　`Variable` 与 `Function`互连并建立一个非循环图，编码完整的计算历史。 每个变量都有一个`.grad_fn` 属性，它引用了一个已经创建了 `Variable`的操作,如加减乘除等（除了用户创建的变量代替`creator is None` 即第一个运算节点, `.grad_fn`为空)  

### Variable和Tensor  
`Tensor`是存在`Variable`中的`.data`里的，而`cpu`和`gpu`的数据是通过 `.cpu()`和`.cuda()`来转换的

In [1]:
import torch
from torch.autograd import Variable

a = Variable(torch.Tensor([1]),requires_grad=True).cuda()

In [2]:
a

tensor([1.], device='cuda:0', grad_fn=<CopyBackwards>)

In [3]:
a.data

tensor([1.], device='cuda:0')

In [4]:
a.cpu()

tensor([1.], grad_fn=<CopyBackwards>)

In [5]:
a.cpu().data

tensor([1.])

In [6]:
a = Variable(torch.Tensor([1]),requires_grad=True) 
b = Variable(torch.Tensor([2]),requires_grad=True)
c = Variable(torch.Tensor([3]),requires_grad=True)

In [7]:
d = a + 2*b
e = d + c

In [8]:
e.backward()

In [9]:
print(a.grad,b.grad,c.grad)

tensor([1.]) tensor([2.]) tensor([1.])


In [10]:
d.grad #中间梯度值不保存，为空

In [11]:
a.grad_fn #第一个节点的.grad_fn为空

In [12]:
e.grad_fn

### 从反向传播中排除子图
　　每个`Variable`都有两个属性，`requires_grad`和`volatile`, 这两个属性都可以将子图从梯度计算中排除并可以增加运算效率
`requires_grad`：排除特定子图，不参与反向传播的计算，即不会累加到`grad`

`volatile`: 推理模式, 计算图中只要有一个子图设置为`True`,　所有子图都会被设置不参与反向传播计算，`.backward()`被禁止

In [13]:
a=Variable(torch.Tensor([1]),requires_grad=False) 
b=Variable(torch.Tensor([2]),requires_grad=True)
c=a+b
c.backward()

In [14]:
a.grad  # 因为a的requires_grad=False 所以不存储梯度值

In [15]:
b.grad

tensor([1.])

In [16]:
a=Variable(torch.Tensor([1]),volatile=True) 
b=Variable(torch.Tensor([2]),requires_grad=True)
c=a+b

/home/weiweia92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """Entry point for launching an IPython kernel.


In [17]:
c.backward()

### 注册钩子：
`Variable`中的`hook`, 相当于插件，可以在既不修改主体的情况下，同时增加额外的功能挂在主体代码上，就好比一个人去打猎，他的衣服上有挂枪的扣子，他可以选择他想要带的枪．来取得不同的打猎效果．　
    
   因此，在`Variable`中通过`register_hook`来实现，`register_hook`的作用是，当反向传播时，你所注册的`hook`都会被调用，比如你可以定义个打印函数，每次反向传播都将`grad值`打印出来．
   
   需要注意的是，`register_hook`函数接收的是一个函数，这个函数有如下的形式：`hook(grad) -> Variable or None`
也就是说，这个函数是拥有改变梯度值的威力的！

In [18]:
import torch
from torch.autograd import Variable

#定义一个打印函数，每次反向传播都将相应的grad打印出来
def print_grad(grad):
    print(grad)

In [19]:
x = Variable(torch.randn([1]), requires_grad=True)

In [20]:
y = x+2

In [21]:
z = torch.mean(torch.pow(y, 2))

In [22]:
y.register_hook(print_grad) #将打印函数挂在变量y上

In [23]:
z.backward()

tensor([5.5940])


In [24]:
y.grad_fn

那一般什么时候我们可以用到注册`hook`呢？有几种常见的情况，比如当我们想要提取中间层参数来进行可视化的时候，或者当我们想要保存中间参数变量，或者我们想要在传播过程中改变梯度值的时候． 

### 自定义Function 

在快速实现想法的过程中，创建自定义的 `Operation` 可以让我们灵活的使用 `pytorch`. 基于这个过程，你所自定义的 `Operation` 需要继承 `class autograd.Function` 类来将其添加到 `autograd` , 这样当我们调用`Operation` 时可以使用 `autograd` 来计算结果和梯度，并编码 `operation` 的历史，在定义过程中每个 `operation` 都需要实现三个方法：

* `__init__（optional)`: 如果你的 `operation` 包含非 `Variable` 的参数，那么可以将其传入到`init` 并在 `operation` 中使用，如果你的 `operation` 不需要额外的参数，你可以忽略`__init__`
* `forward()` : 这里写的是 `operation` 的逻辑代码，可以有任意数量的参数，但参数只能 是`Variable`,返回既可以是 `Variable`, 也可以是 `Variable` 的 `tuple`.
* `backward()`:　梯度计算逻辑代码，参数个数和 `forward()` 返回个数一样，每个参数代表传回到此 `Operation` 的梯度，返回值的个数和此 `operation` 输入的个数一样，如果`operation` 不需要返回梯度，可以返回`None`

注：官方定义一般会用到`@staticmethod` , 同时通过调用`custom_function.apply`来实现,但也可以不加`@staticmethod`,　这样调用要`custom_function()`来实现．

In [32]:
import torch
from torch.autograd import Variable

class custom_add(torch.autograd.Function): 
    """
    forward and backward both use tensor.
    """
    @staticmethod
    def forward(ctx, input, input2):
        """
        input:tensor,output:tensor,ctx是可以用来在反向传播计算的存储属性的对象
        如ctx.save_for_backward可以在backward中使用
        """
        ctx.save_for_backward(input,input2)
        output = input + input2
        return output
    
    @staticmethod
    def backward(ctx,grad_output):
        """
      　backward,我们接受一个存储loss梯度的tensor(grad_output)
        同时根据输入来计算应该返回的的梯度
     　　"""
        input1, input2=ctx.saved_tensors #在forward中存储的数据 save_for_backward
        grad_input=grad_output.clone()
        return grad_input,grad_input #由于input是两个输入，所以也返回两个grad

In [33]:
new_add = custom_add.apply

In [34]:
a=Variable(torch.Tensor([1]),requires_grad=True)
b=Variable(torch.Tensor([2]),requires_grad=True)
c=new_add(a,b)

In [35]:
c

tensor([3.], grad_fn=<custom_addBackward>)

In [36]:
c.backward()

In [37]:
a.grad

tensor([1.])

自定义`function`检查：

在你完成`function`后，你可能会想要知道你的逻辑，反向传播是否有写错，你可以通过比较小数值的差分法结果来进行确认，通过`gradcheck`来实现

In [38]:
from torch.autograd import gradcheck

# gradchek takes a tuple of tensor as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (Variable(torch.Tensor([2]).double(), requires_grad=True), Variable(torch.Tensor([3]).double(), requires_grad=True),)
test = gradcheck(custom_add.apply, input, eps=1e-6, atol=1e-4)

In [39]:
print(test)

True


分析工具：

　　如果你想查看你定义操作的时间花销，`autograd` 的 `profiler` 提供内视每个操作在`GPU`和`CPU`的花销，对于`CPU`通过`profile`, 　基于`nvprof`通过使用`emit_nvtx`

In [40]:
x = Variable(torch.randn(1, 1), requires_grad=True)
with torch.autograd.profiler.profile() as prof:
    y = x ** 2
    y.backward()

In [41]:
print(prof)

-----------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------------------  
Name                                 Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     Number of Calls  Input Shapes                         
-----------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------------------  
pow                                  20.45%           40.499us         20.45%           40.499us         40.499us         1                []                                   
torch::autograd::GraphRoot           1.04%            2.060us          1.04%            2.060us          2.060us          1                []                                   
PowBackward0                         38.66%           76.558us         38.66%           76.558us         76.558us 